In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 62 kB/s 
     |████████████████████████████████| 198 kB 48.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=ea7a23b67caff4ac114862eadafd5a51f77d0169f8baaee56de545e79eac2a29
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [ ]:
import os
import pyspark 
import configparser
import pathlib as pl
from pyspark.sql import SparkSession

In [ ]:
def create_pyspark_session():

  aws_secrets = {
    "PUBLIC_KEY":"",
    "SECRET_KEY":""
  }
  config = configparser.ConfigParser()
  spark_session = (SparkSession
        .builder
        .appName('spark_ifood_challenge')
        .getOrCreate()
      )
  # AWS-S3 connection 
  sc=spark.sparkContext
  hadoop_conf=sc._jsc.hadoopConfiguration()
  hadoop_conf.set("fs.s3n.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
  hadoop_conf.set("fs.s3n.awsAccessKeyId", aws_secrets.get('PUBLIC_KEY'))
  hadoop_conf.set("fs.s3n.awsSecretAccessKey", aws_secrets.get('SECRET_KEY'))

  return spark_session

In [ ]:
def ingest_data(s3_paths, spark:SparkSession):
  for s3_path in s3_paths:

      output_path = "output_path"

      if s3_path.endswith('csv.gz'):
          df = spark.read.csv(s3_path, header=True)
      else:
          continue

      (df.write.format("parquet").mode(write_mode).save(output_path))
      


In [ ]:
def extract_data(spark:SparkSession):
  final_df = (spark \
              .read \
              .format('parquet') \
              .load(self.parquet_filenames[param_filename])
            )
  
  '''
  Add any necessary transformation, if any, with .withColumn
  if we want to load the data into snowflake, after mapping the respective columns, we could use:
    self.snowflake_connection = {
            "sfUrl": 'url',
            "sfUser": 'user',
            "sfPassword": 'password',
            "sfDatabase": 'database',
            "sfSchema": 'schema',
            "sfWarehouse": 'warehouse'
          }

    (final_df\
            .write\
            .format('snowflake')\
            .option('dbtable', target_table_name)\
            .options(**snowflake_connection)\
            .option('column_mapping', 'name')\
            .option('truncate_table', 'on')\
            .option('usestagingtable', 'off')\
            .option('columnmap', column_map) \
            .mode('overwrite')\
            .save()    
        )

  '''

  return df

In [ ]:
files_to_ingest = ['restaurant.csv.gz', 'consumer.csv.gz']
s3_paths = [f's3n://ifood-data-architect-test-source/{f}' for f in files_to_ingest]

FOR THE FILE: restaurant.csv.gz

In [ ]:
spark = create_pyspark_session()

In [ ]:
ingest_data('s3n://ifood-data-architect-test-source/restaurant.csv.gz', spark)

In [ ]:
extract_data(spark)

FOR THE FILE: consumer.csv.gz

In [ ]:
spark = create_pyspark_session()

In [ ]:
ingest_data('s3n://ifood-data-architect-test-source/consumer.csv.gz', spark)

In [ ]:
extract_data(spark)